###### Untuk training intent- agar bisa mengenali dari file input.txt
menjadi sebuah intent yang telah di encoding dengan menggunakan
susunan directory dalam 'data'

In [1]:
#!pip install gensim
#!pip install keras

In [1]:
import codecs
import glob
import re
import nltk
import gensim.models.word2vec as w2v
import numpy as np
import matplotlib.pyplot as plt
import os
import gensim
from keras.models import Sequential
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras import optimizers

C:\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# PARAMETERS

In [3]:
w2vmodel="chat_model1-300.w2v"   #filename
num_features = 300    #number of features in the model
maxsent = 10          #maximum words in a sentence.

### LOAD VECTOR MODEL 

In [4]:
tex2vec =  gensim.models.KeyedVectors.load(w2vmodel)

In [5]:
from chat_wordvec import *

### Cek data anda

swv_ar(twl("irza ada disini")).shape

In [6]:
#test = swv_ar(twl("Saya tidak peduli. Itu terjadi menjadi nama saya.", rempunct=True, flat=True),frontpad=False)
#print(test.shape)
#print(test)

### LOAD DATA ANDA

In [7]:
import glob
import fileinput
fls = glob.glob('./data/*/input.txt')

In [8]:
intent=[]
prep_x = []
prep_y = []
for i in fls:
    for tx in fileinput.input(i):
        print(tx.replace('\n',''), i.split('\\')[-2])
        prep_x.append(tx.replace('\n',''))
        prep_y.append(i.split('\\')[-2])
        intent.append(i.split('\\')[-2])
fileinput.close()

intent = list(set(intent))
intent.sort()
print(intent)

ya affirmasi
iya affirmasi
betul affirmasi
yes affirmasi
setuju affirmasi
oke affirmasi
terima kasih affirmasi
iya affirmasi
terima kasih affirmasi
ho oh affirmasi
laksanakan affirmasi
asik affirmasi
benar affirmasi
lihat jalan cctv_lewatmana
kondisi jalan cctv_lewatmana
saya mau lihat jalan cctv_lewatmana
bagaimana kondisi jalan cctv_lewatmana
lewat mana cctv_lewatmana
lihat cctv cctv_lewatmana
tampilkan cctv cctv_lewatmana
ada apa di jalan cctv_lewatmana
mana jalan yang macet cctv_lewatmana
ada kemacetan cctv_lewatmana
macet dimana cctv_lewatmana
sampai jumpa goodbye
bye goodbye
stop goodbye
udah goodbye
sudah goodbye
terima kasih goodbye
see you goodbye
saya mau lihat gambar lihat_gambar
mau lihat gambar lihat_gambar
lihat gambar lihat_gambar
tolong cari gambar lihat_gambar
tunjukkan gambar lihat_gambar
tunjuk gambar lihat_gambar
kasih lihat gambar lihat_gambar
lihat gambar <mobil> lihat_gambar
lihat gambar <ikan> lihat_gambar
lihat gambar <binatang> lihat_gambar
lihat gambar <kuda>

## Proses step 2 - ambil sinonim

In [9]:
fls2 = glob.glob('./data/*/sinonim.txt')

In [10]:
for ff in fls2:
    with open(ff ,'r') as f:
        tx = f.read()
tx = tx.split(':')

In [11]:
pxx = []
pyy = []
for i in prep_x:
    if i.find(tx[0]) > -1:
        #lihat index dimana di temukan sinonim pertama
        ind = prep_x.index(i)
        for x in range(len(tx)):
            
            #cari dari prep_x mana yg isinya seperti sinonim 1 -> lalu replace
            px = prep_x[prep_x.index(i)].replace(tx[0],tx[x])
            py = prep_y[prep_x.index(i)]
            print(px,py )
            pxx.append(px)
            pyy.append(py)

saya mau lihat gambar lihat_gambar
saya mau lihat lukisan lihat_gambar
saya mau lihat foto lihat_gambar
saya mau lihat skema lihat_gambar
mau lihat gambar lihat_gambar
mau lihat lukisan lihat_gambar
mau lihat foto lihat_gambar
mau lihat skema lihat_gambar
lihat gambar lihat_gambar
lihat lukisan lihat_gambar
lihat foto lihat_gambar
lihat skema lihat_gambar
tolong cari gambar lihat_gambar
tolong cari lukisan lihat_gambar
tolong cari foto lihat_gambar
tolong cari skema lihat_gambar
tunjukkan gambar lihat_gambar
tunjukkan lukisan lihat_gambar
tunjukkan foto lihat_gambar
tunjukkan skema lihat_gambar
tunjuk gambar lihat_gambar
tunjuk lukisan lihat_gambar
tunjuk foto lihat_gambar
tunjuk skema lihat_gambar
kasih lihat gambar lihat_gambar
kasih lihat lukisan lihat_gambar
kasih lihat foto lihat_gambar
kasih lihat skema lihat_gambar
lihat gambar <mobil> lihat_gambar
lihat lukisan <mobil> lihat_gambar
lihat foto <mobil> lihat_gambar
lihat skema <mobil> lihat_gambar
lihat gambar <ikan> lihat_gambar

### Check sama panjangnya

In [13]:
prep_x.extend(pxx)
prep_y.extend(pyy)
print(len(prep_x))
print(len(prep_y))

120
120


### Dense representation one-hot encoding

In [14]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from numpy import argmax

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(intent)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
#print(onehot_encoded)

onehot_encoded[intent.index("goodbye")]
end_dense_layer = len(onehot_encoded)

C:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
train_x=[]
train_y=[]
for i in range(len(prep_x)):
    train_x.append(swv_ar(twl(prep_x[i], rempunct=True, flat=True),frontpad=True))
    train_y.append(onehot_encoded[intent.index(prep_y[i])])

train_x=np.array(train_x)
train_y=np.array(train_y)

print(train_x.shape)
print(train_y.shape)

(120, 10, 300)
(120, 6)


### SIAPKAN DATA

In [16]:
#x_train, x_test, y_train, y_test = train_test_split(RQ, RA, test_size=0.2, random_state=1)

In [17]:
model=Sequential()
model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=False, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(Dense(75, activation="relu"))
model.add(Dense(end_dense_layer,activation="sigmoid"))
#model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
#model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#cosine_proximity
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 300)           721200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_1 (Dense)              (None, 75)                22575     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 456       
Total params: 1,465,431
Trainable params: 1,465,431
Non-trainable params: 0
_________________________________________________________________


### LOADING MODEL SUDAH DI TRAINING

In [19]:
#import keras
#model = keras.models.load_model("model1.chatbot")

# TRAINING MODEL

In [20]:
model.fit(train_x,train_y, epochs=300)
#model.save("model-10.chatbot")

Epoch 1/300
120/120 [==============================] - 1s 7ms/step - loss: -0.4951 - acc: 0.5000
Epoch 2/300
120/120 [==============================] - 0s 2ms/step - loss: -0.5489 - acc: 0.5000
Epoch 3/300
120/120 [==============================] - 0s 2ms/step - loss: -0.5405 - acc: 0.5000
Epoch 4/300
120/120 [==============================] - 0s 2ms/step - loss: -0.5429 - acc: 0.5000
Epoch 5/300
120/120 [==============================] - 0s 2ms/step - loss: -0.5520 - acc: 0.5000
Epoch 6/300
120/120 [==============================] - 0s 2ms/step - loss: -0.5596 - acc: 0.5000
Epoch 7/300
120/120 [==============================] - 0s 2ms/step - loss: -0.5623 - acc: 0.5000
Epoch 8/300
120/120 [==============================] - 0s 2ms/step - loss: -0.5747 - acc: 0.5000
Epoch 9/300
120/120 [==============================] - 0s 2ms/step - loss: -0.5937 - acc: 0.5000
Epoch 10/300
120/120 [==============================] - 0s 2ms/step - loss: -0.6131 - acc: 0.5000
Epoch 11/300
120/120 [=======

120/120 [==============================] - 0s 3ms/step - loss: -0.9935 - acc: 0.9917
Epoch 168/300
120/120 [==============================] - 0s 3ms/step - loss: -0.9934 - acc: 0.9917
Epoch 169/300
120/120 [==============================] - 0s 3ms/step - loss: -0.9935 - acc: 0.9917
Epoch 170/300
120/120 [==============================] - 0s 2ms/step - loss: -0.9933 - acc: 0.9917
Epoch 171/300
120/120 [==============================] - 0s 2ms/step - loss: -0.9933 - acc: 0.9917
Epoch 172/300
120/120 [==============================] - 0s 2ms/step - loss: -0.9935 - acc: 0.9917
Epoch 173/300
120/120 [==============================] - 0s 3ms/step - loss: -0.9934 - acc: 0.9917
Epoch 174/300
120/120 [==============================] - 0s 2ms/step - loss: -0.9934 - acc: 0.9917
Epoch 175/300
120/120 [==============================] - 0s 2ms/step - loss: -0.9933 - acc: 0.9917
Epoch 176/300
120/120 [==============================] - 0s 3ms/step - loss: -0.9932 - acc: 0.9917
Epoch 177/300
120/120 [=